In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
import seaborn as sns
import anndata
import scanpy as sc
import re
import decoupler as dc
import sc_toolbox
import random
import seaborn.objects as so

In [2]:
data_dir = "/well/immune-rep/users/vbw431/Projects/Peppa/data/"
newdata_dir = "/well/immune-rep/users/vbw431/Projects/Peppa/new_analysis/data/"
plot_dir = "/well/immune-rep/users/vbw431/Projects/Peppa/new_out/final_plots/"

sys.path = sys.path + ["/well/immune-rep/users/vbw431/python_utils/"]

In [3]:
import rpy2.rinterface_lib.callbacks
import anndata2ri
import logging
import rpy2.robjects.lib.ggplot2 as gp
from rpy2.robjects import pandas2ri
from rpy2.robjects import r
from rpy2.ipython.ggplot import image_png

#sc.settings.verbosity = 0
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

/well/immune-rep/users/vbw431/conda/ivybridge/envs/scverse_new/lib/python3.8/site-packages/rpy2/robjects/lib/ggplot2.py:70: UserWarning: This was designed againt ggplot2 versions starting with 3.3. but you have 3.4.2
  warnings.warn(


In [4]:
%%R
.libPaths(c(paste0("/well/immune-rep/users/vbw431/conda/ivybridge/envs/scverse_new/r_modules"), .libPaths()))

library(tidyverse)
library(Seurat)


cur.dir = "/well/immune-rep/users/vbw431/Projects/Peppa/data/DIMITRA_FASTQ/"
work.dir = "/well/immune-rep/users/vbw431/Projects/Peppa/"
out.dir = "/well/immune-rep/users/vbw431/Projects/Peppa/new_analysis/out/"
references = "/well/immune-rep/users/vbw431/reference/reference/refdata-gex-GRCh38-2020-A/"
com.out = "/well/immune-rep/users/vbw431/Projects/Peppa/reference_combat/"

##plotting
library(ggplot2); theme_set(theme_bw(base_size = 18)+
                              theme(strip.text = element_text(colour = 'black', face="bold",size=12), 
                                    panel.grid.major = element_blank(), 
                                    panel.grid.minor = element_blank(),
                                    panel.border = element_rect(size = 0.7),
                                    axis.ticks.length=unit(.10, "cm"),
                                    axis.ticks = element_line(size=0.7),
                                    strip.background = element_blank()))



-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.2     v readr     2.1.4
v forcats   1.0.0     v stringr   1.5.0
v lubridate 1.9.2     v tibble    3.2.1
v purrr     1.0.1     v tidyr     1.3.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x purrr::%@%()                 masks rlang::%@%()
x lazyeval::as_name()          masks rlang::as_name()
x lazyeval::call_modify()      masks rlang::call_modify()
x lazyeval::call_standardise() masks rlang::call_standardise()
x lazyeval::expr_label()       masks rlang::expr_label()
x lazyeval::expr_text()        masks rlang::expr_text()
x lazyeval::f_env()            masks rlang::f_env()
x lazyeval::f_env<-()          masks rlang::f_env<-()
x lazyeval::f_label()          masks rlang::f_label()
x lazyeval::f_lhs()            masks rlang::f_lhs()
x lazyeval::f_lhs<-()          masks rlang::f_lhs<-()
x lazyeval::f_rhs()            masks rlang::f_rhs()
x lazyeval::f_rhs<-() 

## Load Seurat data and Prepare for DGE

In [5]:
%%R 

##load, normalise  and trim to remove pre_treatment replicates and post vaccine samples from azimuth

peppa <- readRDS(paste0(out.dir, "/peppa_pbmc_all.rds"))




In [6]:
%%R
peppa_list <- SplitObject(peppa,split.by="Ref_lab")


In [7]:
%%R
septrins_fullnames <- rownames(peppa_list[["Azimuth"]])[grepl("^SEPTIN",rownames(peppa_list[["Azimuth"]]))]

septrins_shortnames <- rownames(peppa_list[["Azimuth"]])[grepl("^SEPT",rownames(peppa_list[["Azimuth"]]))]
septrins_shortnames <- septrins_shortnames[!septrins_shortnames %in% septrins_fullnames]
septrins_shortnames



 [1] "SEPT1"     "SEPT10"    "SEPT11"    "SEPT12"    "SEPT14"    "SEPT2"    
 [7] "SEPT3"     "SEPT4"     "SEPT4-AS1" "SEPT6"     "SEPT7"     "SEPT7-AS1"
[13] "SEPT8"     "SEPT9"    


In [8]:
%%R
septrins_shortnames

 [1] "SEPT1"     "SEPT10"    "SEPT11"    "SEPT12"    "SEPT14"    "SEPT2"    
 [7] "SEPT3"     "SEPT4"     "SEPT4-AS1" "SEPT6"     "SEPT7"     "SEPT7-AS1"
[13] "SEPT8"     "SEPT9"    


In [9]:
%%R 

azimuth <- subset(peppa_list[["Azimuth"]],features=setdiff(rownames(peppa_list[["Azimuth"]]),septrins_fullnames))

azimuth_mat <- GetAssayData(object = azimuth, slot = "counts")
rownames(azimuth_mat) <- gsub("^SEPT","SEPTIN", rownames(azimuth_mat))
newdata_azi <- CreateAssayObject(counts = azimuth_mat)

azimuth_new<- CreateSeuratObject(counts = newdata_azi)

azimuth_new@meta.data <- azimuth@meta.data


In [11]:
%%R
peppa <- subset(peppa_list[["Peppa"]],features=setdiff(rownames(peppa_list[["Peppa"]]),septrins_shortnames))


In [20]:
%%R
pbmc.combined <- merge(peppa, y = azimuth_new)


In [22]:
%%R
peppa <- readRDS(paste0(out.dir, "/peppa_pbmc_all.rds"))


[1]  23789 252811


In [24]:
%%R
dim(peppa) %>% print
dim(pbmc.combined)

[1]  23789 252811
[1]  23776 252811


In [26]:
%%R
saveRDS(pbmc.combined, paste0(out.dir, "/peppa_pbmc_all.rds"))